In [ ]:
#FINAL

# Import all necessary libraries
import webbrowser     # For opening web pages
import time
import re
import requests       # For http request

# Define the starting(seed URL) and current URL
seed_url="https://en.wikipedia.org"
current_url="https://en.wikipedia.org/wiki/Ancient_Indian_sciences"

# Function for  downloading web page
def download_page(url):
  try:
    response=requests.get(url)
    response.raise_for_status()
    return  response.text
  except Exception as e:
    print("ERROR downloading page!!:"+ url +"\n" +str(e))
    print("Either the Wikipedia page does not exist or there is an ERROR in the spelling entered! Try again:")
    return None

# Fuction for parsing URLs
def url_parse(url):
  try:
    from urllib.parse import urlparse
  except ImportError:
    from urlparse import urlparse

  url=url
  s=urlparse(url)
  seed_url_n=seed_url
  i=0
  flag=0

  while i<=9:
    if url=="/":
      url=seed_url_n
      flag=0
    elif not s.scheme:
      url="http://" +url
      flag=0
    elif "#" in url:
      url=url[:url.find("#")]
      flag=0
    elif "?" in url:
      url=url[:url.find("?")]
      flag=0
    elif s.netloc=="":
      url=seed_url+s.path
      flag=0
    elif url[len(url)-1]=="/":
      url=url[:-1]
      flag=0
    else:
      url=url
      flag=0
      break

    i=i+1
    s=urlparse(url)
  return(url,flag)

# Function for opening Wikipedia page Search Results
def open_search_results(keyword):
  search_url="https://en.wikipedia.org/w/index.php?search="+ keyword.replace(' ','+')
  webbrowser.open(search_url)

# Function for extracting title from raw HTML content
def extract_title(page):
  start_title=page.find("<title>")
  end_title=page.find("</title>",start_title)
  if start_title!=-1 and end_title!=-1:
    title=page[start_title + 7:end_title]
    return title
  return "No Title"

# Function for extracting introduction from raw HTML content
def extract_introduction(page):
  start_introduction=page.find("<p>")
  stop_introduction=page.find('<div id="toctitle">',start_introduction +1)
  if '<div id="toctitle">' not in page:
    stop_introduction=page.find('</p>',start_introduction +1)
  else:
    pass

  raw_introduction=page[start_introduction: stop_introduction]
  return raw_introduction

# Function for extracting title from raw HTML content
def extract_links(page):
  links=[]
  while True:
    start_link=page.find("<a href")
    if start_link==-1:
      break
    else:
      start_quote=page.find('"',start_link)
      end_quote=page.find('"',start_quote +1)
      link=page[start_quote +1:end_quote]
      links.append(link)
      page=page[end_quote:]
  return links

# Function for removing HTML tags from the text
def remove_html_tags(text):
  clean_text=re.sub(r'<.+?>','',text)
  return clean_text

# Function for checking File type
def extension_scan(url):
  a=['.png','.jpg','.jpeg','.gif','.tif','.txt']
  j=0
  while j<(len(a)):
    if a[j] in url:
      flag2=1
      break
    else:
      flag2=0
      j=j+1
  return flag2

# Function for the main Crawler part
def web_crawl(start_url,depth=3):
  to_crawl=[start_url]
  database={}
  l=0

  for k in range(0, depth):
    i=0
    crawled=[]
    while i<3:
      if not to_crawl:
        break
      url=to_crawl.pop(0)
      url, flag=url_parse(url)
      # flag=url_parse(url)
      flag2=extension_scan(url)
      time.sleep(3)

      if flag==1 or flag2==1:
        pass
      else:
        l=l+1
        if url in crawled:
          print("link" +str(l)+ ": " +url)
          print("VISITED!!\n")
        else:
          print("Link" +str(l)+ ": " +url)
          raw_html=download_page(url)
          if raw_html:
            title_upper=extract_title(raw_html)
            title=title_upper.lower()
            print("Title = "+title)
            raw_introduction=extract_introduction(raw_html)
            to_crawl=to_crawl+extract_links(raw_introduction)
            crawled.append(url)
            pure_introduction=remove_html_tags(raw_introduction)
            print("Introduction = "+pure_introduction.replace('   ',' '))
            database[title]=pure_introduction

            file=open('database.txt','a')
            file.write(title+ ":" +"\n")
            file.write(pure_introduction+"\n")
            file.close()

            n=1
            j=0
            while j< (len(to_crawl)-n):
              if to_crawl[j] in to_crawl[j+1: (len(to_crawl)-1)]:
                to_crawl.pop(j)
                n=n+1
              else:
                pass
              j=j+1

        i=i+1
        # print(i)
        # print(k)
  return database

# Main loop
while True:
  keyword=input("Press 'Enter-Key' to search the 'Ancient-Indian-Sciences-and-Relics' Wikipedia Page OR\nEnter a search query (or 'q' to quit):").strip()
  if keyword.lower()=='q':
    break
  else:
    # if not keyword.lower():
    if not keyword:
      print("Crawling the DEFAULT page for this search engine !!")
    else:
      current_url="https://en.wikipedia.org/wiki/"+keyword.replace(' ','_')
      open_search_results(keyword)
      print("Wikipedia Crawler started!!")

    t0=time.time()
    database=web_crawl(start_url=current_url,depth=3)
    t1=time.time()
    total_time=t1-t0
    print("Wikipedia Crawler Ended!!")
    print("Total Time taken for the exceution(Time Complexity):",total_time)

    link_stack=[]
    while True:
      print("\n"*2)
      action=input("Entera link number to view details (INTEGER!!) or enter 'q' to quit:").strip()
      if action.lower()=='q':
        break
      elif action.isdigit() and int(action)<=len(database):
        title=list(database.keys())[int(action)-1]
        print("Title:",title)
        print("Introduction:",database[title])
        link_stack.append(list(database.keys())[int(action)-1])
        selected_link=link_stack.pop()
        current_url="https://en.wikipedia.org/wiki/"+ selected_link.replace(' ','_').replace('_-_wikipedia','')
        print("Wikipedia Crawler Started!!")
        open_search_results(selected_link)

        t0=time.time()
        database=web_crawl(start_url=current_url,depth=3)
        t1=time.time()
        total_time=t1-t0
        print("Wikipedia Crawler Ended!!")
        print("Total Time taken for the exceution(Time Complexity):",total_time)
        print("\n"*2)
      else:
        break